In [3]:
import pandas as pd
import numpy as np
from scipy.io import arff
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [69]:
data = arff.loadarff('DryBeanDataset/Dry_Bean_Dataset.arff')
df = pd.DataFrame(data[0])

df['Class'] = df['Class'].str.decode('utf-8')
le = preprocessing.LabelEncoder()
le.fit(df['Class'])
df['Class'] = le.transform(df['Class'])
df.head()

X = tf.convert_to_tensor(df.iloc[:, :-1])

X = tf.divide(
   tf.subtract(
      X, 
      tf.reduce_min(X)
   ), 
   tf.subtract(
      tf.reduce_max(X), 
      tf.reduce_min(X)
   )
)

Y = df.pop('Class')
Y = tf.keras.utils.to_categorical(Y)

In [72]:
Y.shape

(13611, 7)

In [115]:
class BeanModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.dense3 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.dense4 = tf.keras.layers.Dense(7)
        self.softmax1 = tf.keras.layers.Softmax()

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.softmax1(x)
        


In [116]:
model = BeanModel()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer='adam',
                loss=loss_fn,
                metrics=['accuracy'])
model.fit(X, Y, epochs=20, validation_split=0.25)


Epoch 1/20
319/319 [==============================] - 3s 3ms/step - loss: 1.5740 - accuracy: 0.3402 - val_loss: 4.2412 - val_accuracy: 0.0000e+00
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 1.0274 - accuracy: 0.5662 - val_loss: 3.6412 - val_accuracy: 0.0000e+00
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.9348 - accuracy: 0.6044 - val_loss: 2.8907 - val_accuracy: 0.0000e+00
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.9042 - accuracy: 0.6117 - val_loss: 2.7302 - val_accuracy: 0.0000e+00
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.8829 - accuracy: 0.6116 - val_loss: 2.2198 - val_accuracy: 0.0000e+00
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.8642 - accuracy: 0.6121 - val_loss: 2.5895 - val_accuracy: 0.0000e+00
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.8488 - accuracy: 0.6234 - val_loss: 2.2074 - val

In [82]:
from sklearn.metrics import classification_report

In [96]:
Y_actual_report = np.argmax(Y, axis=1)
Y_actual_report

array([5, 5, 5, ..., 3, 3, 3], dtype=int64)

In [97]:
Y_train_actual = np.argmax(model(X), axis=1)
Y_train_actual

array([5, 5, 5, ..., 6, 6, 6], dtype=int64)

In [98]:
print(classification_report(Y_actual_report, Y_train_actual))

              precision    recall  f1-score   support

           0       0.41      0.31      0.35      1322
           1       1.00      1.00      1.00       522
           2       0.64      0.69      0.66      1630
           3       1.00      0.11      0.20      3546
           4       0.61      0.59      0.60      1928
           5       0.24      0.51      0.33      2027
           6       0.53      0.78      0.63      2636

    accuracy                           0.49     13611
   macro avg       0.63      0.57      0.54     13611
weighted avg       0.64      0.49      0.46     13611



In [99]:
from sklearn.decomposition import PCA


In [103]:
pca = PCA(whiten=True)
X_pca = pca.fit_transform(X)


In [118]:
model_with_pca = BeanModel()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model_with_pca.compile(optimizer='adam',
                loss=loss_fn,
                metrics=['accuracy'])
model_with_pca.fit(X_pca, Y, epochs=20, validation_split=0.25)

Epoch 1/20
319/319 [==============================] - 1s 3ms/step - loss: 0.5993 - accuracy: 0.8276 - val_loss: 4.7022 - val_accuracy: 0.0594
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1523 - accuracy: 0.9542 - val_loss: 4.7639 - val_accuracy: 0.1845
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1403 - accuracy: 0.9565 - val_loss: 5.1887 - val_accuracy: 0.2010
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1329 - accuracy: 0.9576 - val_loss: 5.3259 - val_accuracy: 0.2169
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1281 - accuracy: 0.9588 - val_loss: 5.0559 - val_accuracy: 0.2501
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1256 - accuracy: 0.9594 - val_loss: 5.6276 - val_accuracy: 0.2413
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1232 - accuracy: 0.9602 - val_loss: 5.8653 - val_accuracy: 0.2421
Epoch 